In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# Set variable to image directory
img_dir = '/content/drive/My Drive/CelebA/Img/img_align_celeba_6500'
check_save_dir = '/content/drive/My Drive/CelebA/Checkpoints'

# Define list of class names (subset of the classes listed in `list_attr_celeba.txt`)
class_names = ["Arched_Eyebrows", "Bags_Under_Eyes", "Bangs", "Black_Hair", "Blond_Hair", "Brown_Hair", "Eyeglasses", "Gray_Hair", "Heavy_Makeup", "High_Cheekbones", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "Rosy_Cheeks", "Smiling", "Straight_Hair", "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace"]

# Define df as a dataframe from directory 
df=pd.read_csv('/content/drive/My Drive/CelebA/Anno/labels.csv', header=0, delimiter=',', usecols=None, names=["filenames", "Arched_Eyebrows", "Bags_Under_Eyes", "Bangs", "Black_Hair", "Blond_Hair", "Brown_Hair", "Eyeglasses", "Gray_Hair", "Heavy_Makeup", "High_Cheekbones", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "Rosy_Cheeks", "Smiling", "Straight_Hair", "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace"])
print(df)

from sklearn.model_selection import train_test_split
train, valid = train_test_split(df, test_size = 0.3)
valid, test = train_test_split(valid, test_size=0.2)
train_features = train[["Arched_Eyebrows", "Bags_Under_Eyes", "Bangs", "Black_Hair", "Blond_Hair", "Brown_Hair", "Eyeglasses", "Gray_Hair", "Heavy_Makeup", "High_Cheekbones", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "Rosy_Cheeks", "Smiling", "Straight_Hair", "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace"]]
print(train,valid,test)

In [ ]:
from tensorflow import keras
%pip install git+https://github.com/keras-team/keras-preprocessing.git
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.layers.experimental import preprocessing

import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
# Reproducability
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    #os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

# Load training and validation sets
datagen=ImageDataGenerator(rescale=1./255., 
                          rotation_range=20,
                          horizontal_flip=True,
                          vertical_flip=True,
                          width_shift_range=0.2,
                          height_shift_range=0.2,)
valid_datagen=ImageDataGenerator(rescale=1./255.,
                                rotation_range=20,
                                horizontal_flip=True,
                                vertical_flip=True,
                                width_shift_range=0.2,
                                height_shift_range=0.2,)
test_datagen=ImageDataGenerator(rescale=1./255.)

train_generator = datagen.flow_from_dataframe(
    dataframe=train,
    directory=img_dir,
    x_col='filenames',
    y_col=class_names,
    save_format='jpg',
    batch_size=35,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(128, 128)
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid,
    directory=img_dir,
    x_col='filenames',
    y_col=class_names,
    save_format='jpg',
    batch_size=30,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(128, 128)
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test,
    directory=img_dir,
    x_col='filenames',
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(128, 128)
)

# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 128, 128, 3], [None, len(class_names)])
).map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)

ds_valid = tf.data.Dataset.from_generator(
    lambda: valid_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 128, 128, 3], [None, len(class_names)])
).map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)

ds_test = tf.data.Dataset.from_generator(
    lambda: test_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 128, 128, 3], [None, len(class_names)])
).map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
ds_test = tf.data.Dataset.from_generator(
    lambda: test_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 128, 128, 3], [None, len(class_names)])
).map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
import tensorflow.keras.backend as K
K.clear_session()

In [ ]:
inputs = tf.keras.layers.Input(shape=(128, 128, 3))
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
output1 = tf.keras.layers.Dense(1, activation='sigmoid', name='Arched_Eyebrows')(x)
output2 = tf.keras.layers.Dense(1, activation='sigmoid', name='Bags_Under_Eyes')(x)
output3 = tf.keras.layers.Dense(1, activation='sigmoid', name='Bangs')(x)
output4 = tf.keras.layers.Dense(1, activation='sigmoid', name='Black_Hair')(x)
output5 = tf.keras.layers.Dense(1, activation='sigmoid', name='Blond_Hair')(x)
output6 = tf.keras.layers.Dense(1, activation='sigmoid', name='Brown_Hair')(x)
output7 = tf.keras.layers.Dense(1, activation='sigmoid', name='Eyeglasses')(x)
output8 = tf.keras.layers.Dense(1, activation='sigmoid', name='Gray_Hair')(x)
output9 = tf.keras.layers.Dense(1, activation='sigmoid', name='Heavy_Makeup')(x)
output10 = tf.keras.layers.Dense(1, activation='sigmoid', name='High_Cheekbones')(x)
output11 = tf.keras.layers.Dense(1, activation='sigmoid', name='Mouth_Slightly_Open')(x)
output12 = tf.keras.layers.Dense(1, activation='sigmoid', name='Mustache')(x)
output13 = tf.keras.layers.Dense(1, activation='sigmoid', name='Narrow_Eyes')(x)
output14 = tf.keras.layers.Dense(1, activation='sigmoid', name='Rosy_Cheeks')(x)
output15 = tf.keras.layers.Dense(1, activation='sigmoid', name='Smiling')(x)
output16 = tf.keras.layers.Dense(1, activation='sigmoid', name='Straight_Hair')(x)
output17 = tf.keras.layers.Dense(1, activation='sigmoid', name='Wavy_Hair')(x)
output18 = tf.keras.layers.Dense(1, activation='sigmoid', name='Wearing_Earrings')(x)
output19 = tf.keras.layers.Dense(1, activation='sigmoid', name='Wearing_Hat')(x)
output20 = tf.keras.layers.Dense(1, activation='sigmoid', name='Wearing_Lipstick')(x)
output21 = tf.keras.layers.Dense(1, activation='sigmoid', name='Wearing_Necklace')(x)

model = tf.keras.models.Model(inputs=inputs, outputs=[
                                                      output1,
                                                      output2,
                                                      output3,
                                                      output4,
                                                      output5,
                                                      output6,
                                                      output7,
                                                      output8,
                                                      output9, 
                                                      output10,
                                                      output11,
                                                      output12,
                                                      output13,
                                                      output14,
                                                      output15,
                                                      output16,
                                                      output17,
                                                      output18,
                                                      output19,
                                                      output20,
                                                      output21
                                                      ])


In [ ]:
model.compile(optimizer='adam',
              loss={'Arched_Eyebrows': 'binary_crossentropy', 
              'Bags_Under_Eyes': 'binary_crossentropy',
              'Bangs': 'binary_crossentropy',
              'Black_Hair': 'binary_crossentropy',
              'Blond_Hair': 'binary_crossentropy',
              'Brown_Hair': 'binary_crossentropy',
              'Eyeglasses': 'binary_crossentropy',
              'Gray_Hair': 'binary_crossentropy',
              'Heavy_Makeup': 'binary_crossentropy',
              'High_Cheekbones': 'binary_crossentropy',
              'Mouth_Slightly_Open': 'binary_crossentropy',
              'Mustache': 'binary_crossentropy',
              'Narrow_Eyes': 'binary_crossentropy',
              'Rosy_Cheeks': 'binary_crossentropy',
              'Smiling': 'binary_crossentropy',
              'Straight_Hair': 'binary_crossentropy',
              'Wavy_Hair': 'binary_crossentropy',
              'Wearing_Earrings': 'binary_crossentropy',
              'Wearing_Hat': 'binary_crossentropy',
              'Wearing_Lipstick': 'binary_crossentropy',
              'Wearing_Necklace': 'binary_crossentropy'
              },
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

checkpoint = ModelCheckpoint(
    check_save_dir,
    save_weights_only=True,
    save_best_only=True,
    monitor='loss',
    mode='min',
    verbose=1
)

# define early stopping callback
early_stopping = EarlyStopping(
    monitor='loss', # metric to monitor for early stopping
    patience=5, # number of epochs to wait before stopping
    restore_best_weights=True, # restore the best model weights found during training
)

In [ ]:
def generator_wrapper(dataset):
    for batch_x, batch_y in dataset:
        yield (batch_x, [batch_y[:, i] for i in range(21)])
        
ds_train_wrapped = generator_wrapper(ds_train)
ds_valid_wrapped = generator_wrapper(ds_valid)

In [ ]:
%cd drive/My Drive/CelebA/Checkpoints

In [ ]:
history = model.fit(
                    ds_train_wrapped, 
                    epochs=20, 
                    validation_data=ds_valid_wrapped, 
                    batch_size=35, 
                    steps_per_epoch=35, 
                    validation_steps=30,
                    verbose=1,
                    callbacks=[checkpoint, early_stopping])

In [ ]:
%cd ../Saved_models

In [ ]:
tf.keras.saving.save_model(
    model, 'model_01.tf', overwrite=False, save_format='tf',
)

In [ ]:
# Evaluate the model on the test data
def test_generator_wrapper(dataset):
    for batch_x, batch_y in dataset:
        yield (batch_x, tuple(batch_y[:, i] for i in range(21)))

ds_test_wrapped = test_generator_wrapper(ds_test)
test_loss, test_metrics = model.evaluate(ds_test_wrapped)

In [ ]:
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore")
plt.xlabel('Epochs')
plt.ylabel('Val_Accuracy')

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, [ 
                  'val_Arched_Eyebrows_accuracy',
                  'val_Bags_Under_Eyes_accuracy',
                  'val_Bangs_accuracy',
                  'val_Black_Hair_accuracy',
                  'val_Blond_Hair_accuracy',
                     ]].plot()
history_frame.loc[:, [
                  'val_Brown_Hair_accuracy',
                  'val_Eyeglasses_accuracy',
                  'val_Gray_Hair_accuracy',
                  'val_Heavy_Makeup_accuracy',
                  'val_High_Cheekbones_accuracy',
                     ]].plot()
history_frame.loc[:, [
                  'val_Mouth_Slightly_Open_accuracy',
                  'val_Mustache_accuracy',
                  'val_Narrow_Eyes_accuracy',
                  'val_Rosy_Cheeks_accuracy',
                  'val_Smiling_accuracy',
                     ]].plot()
history_frame.loc[:, [
                  'val_Straight_Hair_accuracy',
                  'val_Wavy_Hair_accuracy',
                  'val_Wearing_Earrings_accuracy',
                  'val_Wearing_Hat_accuracy',
                  'val_Wearing_Lipstick_accuracy',
                  'val_Wearing_Necklace_accuracy',
                  ]].plot()